# Accessing storage in Databricks
---
## 0. Prerequisites
### 0.1. Azure App registration

1. [Create an Azure AD application and service principal that can access resources](https://docs.microsoft.com/en-us/azure/azure-resource-manager/resource-group-create-service-principal-portal). Note the following properties:
  - application-id: An ID that uniquely identifies the application
  - directory-id: An ID that uniquely identifies the Azure AD instance
  - storage-account-name: The name of the storage account
  - service-credential: A string that the application uses to prove its identity   
  
2. Register the service principal, granting the correct role assignment, such as Storage Blob Data Contributor, on the Azure Data Lake Storage Gen2 account.

### 0.2. Adding scoped secrets
To add a secret and a scope, this needs to be completed using the [Databricks CLI](https://docs.databricks.com/dev-tools/cli/index.html).

Once signed in via the CLI, issue the following commands:
1. `databricks secrets create-scope --scope Analysts`
2. `databricks secrets put --scope Analysts --key SPID --string-value "Service Principal ID"` (Application Client ID)
3. `databricks secrets put --scope Analysts --key SPKey --string-value "Service Principal Secret Key"`
4. `databricks secrets put --scope Analysts --key DirectoryID --string-value "Azure Directory ID"`

> Update: This can also be done via the browser via the URL **https://<\location>.azuredatabricks.net/?o=<\orgID>#secrets/createScope**

## 1. Mount Azure Data Lake Storage Gen2 filesystem
This only needs to be done once per cluster

In [4]:
dbutils.fs.unmount('/mnt/ecovacs')
dbutils.fs.unmount('/mnt/transactions')

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-2477227571656380> in <module> 
 ----> 1 dbutils . fs . unmount ( '/mnt/ecovacs' ) 
 2 dbutils . fs . unmount ( '/mnt/transactions' ) 

 /local_disk0/tmp/1604282408050-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 312 exc . __context__ = None 
 313 exc . __cause__ = None 
 --> 314 raise exc
 315 return f_with_exception_handling
 316 

 ExecutionError : An error occurred while calling o357.unmount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: Directory not mounted: /mnt/ecovacs; nested exception is: 
	java.lang.IllegalArgumentException: Directory not mounted: /mnt/ecovacs
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:123)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:63)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.unmount(DBUtilsCore.scala:557)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.unmount(DBUtilsCore.scala:568)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: Directory not mounted: /mnt/ecovacs
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.deleteMount(MetadataManager.scala:270)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:90)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:97)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:96)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:96)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:357)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:325)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$2(ServerBackend.scala:53)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:80)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:80)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$1(ServerBackend.scala:49)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$4(UsageLogging.scala:432)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:240)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:235)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:232)
	at com.databricks.rpc.ServerBackend.withAttributionContext(ServerBackend.scala:16)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:277)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:270)
	at com.databricks.rpc.ServerBackend.withAttributionTags(ServerBackend.scala:16)
	at com.databricks.logging.UsageLogging.recordOperation(UsageLogging.scala:413)
	at com.databricks.logging.UsageLog

In [5]:
# service principal details
client_id     = dbutils.secrets.get(scope='Analysts', key='SPID')
client_secret = dbutils.secrets.get(scope='Analysts', key='SPKey')
directory_id  = dbutils.secrets.get(scope='Analysts', key='DirectoryID')

# data lake container details
account_name = 'storagejamesleslie'
container_name = 'ecovacs'

configs = {'fs.azure.account.auth.type': 'OAuth',
           'fs.azure.account.oauth.provider.type': 'org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider',
           'fs.azure.account.oauth2.client.id': client_id,
           'fs.azure.account.oauth2.client.secret': client_secret,
           'fs.azure.account.oauth2.client.endpoint': f'https://login.microsoftonline.com/{directory_id}/oauth2/token'}

# Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
    source = f'abfss://{container_name}@{account_name}.dfs.core.windows.net/',
    mount_point = f'/mnt/{container_name}',
    extra_configs = configs)

# list all items in container, use display() for nicer formatting
display(dbutils.fs.ls(f'/mnt/{container_name}'))

path,name,size
dbfs:/mnt/ecovacs/BestBuy/,BestBuy/,0
dbfs:/mnt/ecovacs/VSI/,VSI/,0


### Unmount a mount point

In [7]:
# unmount container
dbutils.fs.unmount(f'/mnt/{container_name}')

/mnt/ecovacs has been unmounted.
Out[10]: True

## 2. Access directly with service principal and OAuth 2.0
All code below needs to be run every time

In [9]:
# service principal details
client_id     = dbutils.secrets.get(scope='Analysts', key='SPID')
client_secret = dbutils.secrets.get(scope='Analysts', key='SPKey')
directory_id  = dbutils.secrets.get(scope='Analysts', key='DirectoryID')

# update spark configs
spark.conf.set('fs.azure.account.auth.type', 'OAuth')
spark.conf.set('fs.azure.account.oauth.provider.type', 'org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider')
spark.conf.set('fs.azure.account.oauth2.client.id', client_id)
spark.conf.set('fs.azure.account.oauth2.client.secret', client_secret)
spark.conf.set('fs.azure.account.oauth2.client.endpoint', f'https://login.microsoftonline.com/{directory_id}/oauth2/token')

# list all items in container / directory
display(dbutils.fs.ls('abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/'))

path,name,size
abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/archive/,archive/,0
abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/delta/,delta/,0
abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/errors/,errors/,0
abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/processed/,processed/,0


---
# Using dbutils

In [11]:
dbutils.fs.help()

dbutils.fs provides utilities for working with FileSystems. Most methods in
this package can take either a DBFS path (e.g., "/foo" or "dbfs:/foo"), or
another FileSystem URI.

For more info about a method, use dbutils.fs.help("methodName") .

In notebooks, you can also use the %fs shorthand to access DBFS. The %fs shorthand maps
straightforwardly onto dbutils calls. For example, "%fs head --maxBytes=10000 /file/path"
translates into "dbutils.fs.head("/file/path", maxBytes = 10000)".
 fsutils cp(from: String, to: String, recurse: boolean = false): boolean -> Copies a file or directory, possibly across FileSystems head(file: String, maxBytes: int = 65536): String -> Returns up to the first 'maxBytes' bytes of the given file as a String encoded in UTF-8 ls(dir: String): Seq -> Lists the contents of a directory mkdirs(dir: String): boolean -> Creates the given directory if it does not exist, also creating any necessary parent directories mv(from: String, to: String, recurse: boolean = false): boolean -> Moves a file or directory, possibly across FileSystems put(file: String, contents: String, overwrite: boolean = false): boolean -> Writes the given String out to a file, encoded in UTF-8 rm(dir: String, recurse: boolean = false): boolean -> Removes a file or directory mount mount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Mounts the given source directory into DBFS at the given mount point mounts: Seq -> Displays information about what is mounted within DBFS refreshMounts: boolean -> Forces all machines in this cluster to refresh their mount cache, ensuring they receive the most recent information unmount(mountPoint: String): boolean -> Deletes a DBFS mount point

## 1. Listing files / directories

In [13]:
# Mount the "transactions" container
dbutils.fs.mount(
    source = f'abfss://transactions@{account_name}.dfs.core.windows.net/',
    mount_point = f'/mnt/transactions',
    extra_configs = configs)

# list all mounted directories
display(dbutils.fs.ls('/mnt'))

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-460832265447207> in <module> 
 3 source = f'abfss://transactions@{account_name}.dfs.core.windows.net/' , 
 4 mount_point = f'/mnt/transactions' , 
 ----> 5 extra_configs = configs)
 6 
 7 # list all mounted directories 

 /local_disk0/tmp/1604282408050-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 312 exc . __context__ = None 
 313 exc . __cause__ = None 
 --> 314 raise exc
 315 return f_with_exception_handling
 316 

 ExecutionError : An error occurred while calling o357.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/transactions; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/transactions
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:123)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:63)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:465)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/transactions
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$insertMount$1(MetadataManager.scala:220)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:322)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:216)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:77)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:97)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:96)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:96)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:357)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:325)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$2(ServerBackend.scala:53)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:80)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:80)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$1(ServerBackend.scala:49)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$4(UsageLogging.scala:432)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:240)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:235)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:232)
	at com.databricks.

In [14]:
# list all items in the 2020 directory
display(dbutils.fs.ls('/mnt/transactions/2020'))

path,name,size
dbfs:/mnt/transactions/2020/p01/,p01/,0
dbfs:/mnt/transactions/2020/p02/,p02/,0
dbfs:/mnt/transactions/2020/p03/,p03/,0
dbfs:/mnt/transactions/2020/p04/,p04/,0
dbfs:/mnt/transactions/2020/p05/,p05/,0
dbfs:/mnt/transactions/2020/p06/,p06/,0
dbfs:/mnt/transactions/2020/p07/,p07/,0
dbfs:/mnt/transactions/2020/p08/,p08/,0
dbfs:/mnt/transactions/2020/p09/,p09/,0
dbfs:/mnt/transactions/2020/p10/,p10/,0


## 2. Creating new directories

In [16]:
# create a new directory
dbutils.fs.mkdirs('/mnt/transactions/2020/p14')

# check that it appears in the mount point
display(dbutils.fs.ls('/mnt/transactions/2020'))

path,name,size
dbfs:/mnt/transactions/2020/p01/,p01/,0
dbfs:/mnt/transactions/2020/p02/,p02/,0
dbfs:/mnt/transactions/2020/p03/,p03/,0
dbfs:/mnt/transactions/2020/p04/,p04/,0
dbfs:/mnt/transactions/2020/p05/,p05/,0
dbfs:/mnt/transactions/2020/p06/,p06/,0
dbfs:/mnt/transactions/2020/p07/,p07/,0
dbfs:/mnt/transactions/2020/p08/,p08/,0
dbfs:/mnt/transactions/2020/p09/,p09/,0
dbfs:/mnt/transactions/2020/p10/,p10/,0


In [17]:
# does it appear in the data lake too?
display(dbutils.fs.ls('abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/'))

path,name,size
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p01/,p01/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p02/,p02/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p03/,p03/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p04/,p04/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p05/,p05/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p06/,p06/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p07/,p07/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p08/,p08/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p09/,p09/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p10/,p10/,0


## 3. Remove directory

In [19]:
# remove the new directory and all its contents
dbutils.fs.rm('/mnt/transactions/2020/p14', recurse=True)

# has it been removed from the data laker too?
display(dbutils.fs.ls('abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/'))

path,name,size
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p01/,p01/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p02/,p02/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p03/,p03/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p04/,p04/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p05/,p05/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p06/,p06/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p07/,p07/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p08/,p08/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p09/,p09/,0
abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p10/,p10/,0


## Query a SQL Server Database

In [21]:
jdbcHostname = '<server name>.database.windows.net'
jdbcDatabase = '<database name>'
userName = '<user name>'
password = '<password>'  # store this in a Databricks secret scope
jdbcPort = 1433
jdbcUrl = f'jdbc:sqlserver://{jdbcHostname}:{jdbcPort};database={jdbcDatabase};user={userName};password={password}'

df = spark.read.jdbc(url=jdbcUrl, table='<table name>')
display(df)